# Preppin Data Challenge - 2024 Week 2 - Average Price Analysis

###### Importing modules and bringing in the data

In [1]:
import pandas as pd
import numpy as np

###### The dataframes, flowcard_data and non_flowcard_data, are the output files from the PD Week 1 Challenge

In [2]:
flowcard_data = pd.read_csv('/home/lyon/Desktop/Python Projects/Data Cleaning/Preppin Data - 2024/Input Files/flowcard_data')

flowcard_data.head()

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type
0,2024-07-22,PA010,Tokyo,New York,Economy,2380.0,Yes,0,Egg Free
1,2024-04-20,PA002,New York,London,Economy,3490.0,Yes,1,Vegan
2,2024-01-23,PA010,Tokyo,New York,Premium Economy,825.0,Yes,1,Vegetarian
3,2024-06-05,PA006,Tokyo,London,First Class,618.0,Yes,3,Vegan
4,2024-03-30,PA004,Perth,London,First Class,446.0,Yes,1,Nut Free


In [3]:
non_flowcard_data = pd.read_csv('/home/lyon/Desktop/Python Projects/Data Cleaning/Preppin Data - 2024/Input Files/non_flowcard_data')

non_flowcard_data.head()

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type
0,2024-09-28,PA008,Perth,New York,Economy,1855.0,No,2,Vegetarian
1,2024-10-01,PA008,Perth,New York,Business Class,634.8,No,0,Vegetarian
2,2024-03-04,PA007,New York,Perth,Business Class,458.4,No,3,Nut Free
3,2024-02-25,PA010,Tokyo,New York,Premium Economy,1435.0,No,0,None
4,2024-03-29,PA004,Perth,London,Economy,2730.0,No,2,Vegan


###### Unioning the two data frames 

In [4]:
dataframes = [flowcard_data, non_flowcard_data]

complete_data = pd.concat(dataframes)

complete_data.head()

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type
0,2024-07-22,PA010,Tokyo,New York,Economy,2380.0,Yes,0,Egg Free
1,2024-04-20,PA002,New York,London,Economy,3490.0,Yes,1,Vegan
2,2024-01-23,PA010,Tokyo,New York,Premium Economy,825.0,Yes,1,Vegetarian
3,2024-06-05,PA006,Tokyo,London,First Class,618.0,Yes,3,Vegan
4,2024-03-30,PA004,Perth,London,First Class,446.0,Yes,1,Nut Free


In [5]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3778 entries, 0 to 1894
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           3778 non-null   object 
 1   Flight Number  3778 non-null   object 
 2   From           3778 non-null   object 
 3   To             3778 non-null   object 
 4   Class          3778 non-null   object 
 5   Price          3778 non-null   float64
 6   Flow Card?     3778 non-null   object 
 7   Bags Checked   3778 non-null   int64  
 8   Meal Type      3778 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 295.2+ KB


###### Creating a new column, Quarter, which is the date part of Quarter that is extracted from the Date column

In [6]:
complete_data['Quarter'] = pd.to_datetime(complete_data['Date'])

complete_data['Quarter'] = complete_data['Quarter'].dt.quarter

complete_data['Quarter'].head()

0    3
1    2
2    1
3    2
4    1
Name: Quarter, dtype: int64

In [7]:
complete_data.head()

,Date,Flight Number,From,To,Class,Price,Flow Card?,Bags Checked,Meal Type,Quarter
0,2024-07-22,PA010,Tokyo,New York,Economy,2380.0,Yes,0,Egg Free,3
1,2024-04-20,PA002,New York,London,Economy,3490.0,Yes,1,Vegan,2
2,2024-01-23,PA010,Tokyo,New York,Premium Economy,825.0,Yes,1,Vegetarian,1
3,2024-06-05,PA006,Tokyo,London,First Class,618.0,Yes,3,Vegan,2
4,2024-03-30,PA004,Perth,London,First Class,446.0,Yes,1,Nut Free,1


###### Removing unnecessary columns. Resulting dataframe, cleaned_data, is ready to go through the reshaping process

In [8]:
cleaned_data = complete_data.drop(columns=['Date', 'Flight Number', 'From', 'To', 'Bags Checked', 'Meal Type'])

cleaned_data = cleaned_data[['Flow Card?', 'Quarter', 'Class', 'Price']]

cleaned_data.head()

,Flow Card?,Quarter,Class,Price
0,Yes,3,Economy,2380.0
1,Yes,2,Economy,3490.0
2,Yes,1,Premium Economy,825.0
3,Yes,2,First Class,618.0
4,Yes,1,First Class,446.0


In [9]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3778 entries, 0 to 1894
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Flow Card?  3778 non-null   object 
 1   Quarter     3778 non-null   int64  
 2   Class       3778 non-null   object 
 3   Price       3778 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 147.6+ KB


# This section of the notebook handles the process of reshaping the dataframe of cleaned_data into the final dataframe of final_data

### The logic of this section is: 

#### 1: Create three dataframes which represent the flight sales data for each flight class (Economy, Premium, Business and First) by each price aggregation (Minimum, Median and Maximum) --- in total, there should be 12 dataframes 

#### 2: Rename the Class column for the 12 dataframes to reflect the actual flight class they correspond to (see the instructions from the challenge webpage)

#### 3: Go from 12 dataframes to just four dataframes // union the dataframes based on flight class (for example, the dataframes representing the minimum, median and maximum prices for Business flights all get consolidated into one dataframe called data_for_business)

#### 4: Rename the Price column for each consolidated dataframe (for example, for data_for_business, Price gets renamed to Business)

#### 5: Add an index column for each of the consolidated dataframes (this is needed for steps 6 and 7)

#### 6. Create two intermediary dataframes, which represent joining two consolidated dataframes into one intermediate dataframe (for example, joining data_for_economy and data_for_premium together to get the dataframe reshaped_economy_and_premium_data)

#### 7: Join the 2 intermediary dataframes together and drop unnecessary columns. This creates the final dataframe, final_data, which represents the desired output for this challenge

##### Creating the data_for_premium dataframe (steps 1 to 5)

In [10]:
# Business to Premium

business_class = cleaned_data.query('Class == "Business Class"')

# step 1 

business_class_minimum_price = business_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).min()

business_class_median_price = business_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).median()

business_class_maximum_price = business_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).max()

# step 2

premium_class_minimum_price = business_class_minimum_price.replace(['Economy'], 'Premium')

premium_class_median_price = business_class_median_price.replace(['Economy'], 'Premium')

premium_class_maximum_price = business_class_maximum_price.replace(['Economy'], 'Premium')

#

# this code unions the three dataframes which represent the minimum, median and maximum price dataframes 
# for Premium class flights

# this dataframe will then be unioned with the other classes finalized dataframes and then pivoted
# to make the final result table 

# steps 3 to 5

dataframes = [premium_class_minimum_price, premium_class_median_price, premium_class_maximum_price]

data_for_premium = pd.concat(dataframes)

data_for_premium['index'] = np.arange(len(data_for_premium))

data_for_premium = data_for_premium.rename(columns={'Price':'Premium'})

data_for_premium.drop(columns=['Class'], inplace=True)
                                        
data_for_premium.head()

# Business Class to Premium metrics and conversion is done

,Flow Card?,Quarter,Premium,index
0,No,1,241.2,0
1,No,2,240.0,1
2,No,3,240.0,2
3,No,4,240.0,3
4,Yes,1,249.6,4


###### Creating the data_for_first dataframe (steps 1 to 5)

In [11]:
# Economy to First

economy_class = cleaned_data.query('Class == "Economy"')

# step 1 

economy_class_minimum_price = economy_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).min()

economy_class_median_price = economy_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).median()

economy_class_maximum_price = economy_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).max()

# step 2

first_class_minimum_price = economy_class_minimum_price.replace(['Economy'], 'First')

first_class_median_price = economy_class_median_price.replace(['Economy'], 'First')

first_class_maximum_price = economy_class_maximum_price.replace(['Economy'], 'First')

# steps 3 to 5

dataframes = [first_class_minimum_price, first_class_median_price, first_class_maximum_price]

data_for_first = pd.concat(dataframes)

data_for_first['index'] = np.arange(len(data_for_first))

data_for_first = data_for_first.rename(columns={'Price':'First'})

data_for_first.drop(columns=['Class'], inplace=True)

data_for_first.head()

# Economy to First class metrics and conversion is done


,Flow Card?,Quarter,First,index
0,No,1,1030.0,0
1,No,2,1000.0,1
2,No,3,1000.0,2
3,No,4,1015.0,3
4,Yes,1,1020.0,4


###### Creating the data_for_economy dataframe (steps 1 to 5)

In [12]:
# First Class to Economy

first_class = cleaned_data.query('Class == "First Class"')

# step 1 

first_class_minimum_price = first_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).min()

first_class_median_price = first_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).median()

first_class_maximum_price = first_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).max()

# step 2

economy_class_minimum_price = first_class_minimum_price.replace(['First Class'], 'Economy')

economy_class_median_price = first_class_median_price.replace(['First Class'], 'Economy')

economy_class_maximum_price = first_class_maximum_price.replace(['First Class'], 'Economy')

# steps 3 to 5

dataframes = [economy_class_minimum_price, economy_class_median_price, economy_class_maximum_price]

data_for_economy = pd.concat(dataframes)

data_for_economy['index'] = np.arange(len(data_for_economy))

data_for_economy = data_for_economy.rename(columns={'Price':'Economy'})

data_for_economy.drop(columns=['Class'], inplace=True)

data_for_economy.head()

# First class to Economy class metrics and conversion is done

,Flow Card?,Quarter,Economy,index
0,No,1,204.0,0
1,No,2,202.0,1
2,No,3,201.0,2
3,No,4,200.0,3
4,Yes,1,201.0,4


###### Creating the data_for_business dataframe (steps 1 to 5)

In [13]:
# Premium Economy to Business

premium_class = cleaned_data.query('Class == "Premium Economy"')

# step 1 

premium_class_minimum_price = premium_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).min()

premium_class_median_price = premium_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).median()

premium_class_maximum_price = premium_class.groupby(['Flow Card?', 'Quarter', 'Class'], as_index=False).max()

# step 2

business_class_minimum_price = premium_class_minimum_price.replace(['Premium Economy'], 'Business')

business_class_median_price = premium_class_median_price.replace(['Premium Economy'], 'Business')

business_class_maximum_price = premium_class_maximum_price.replace(['Premium Economy'], 'Business')


# steps 3 to 5

dataframes = [business_class_minimum_price, business_class_median_price, business_class_maximum_price]

data_for_business = pd.concat(dataframes)

data_for_business['index'] = np.arange(len(data_for_business))

data_for_business = data_for_business.rename(columns={'Price':'Business'})

data_for_business.drop(columns=['Class'], inplace=True)

data_for_business.head()

# Premium Economy class to Business class metrics and conversion is done

,Flow Card?,Quarter,Business,index
0,No,1,515.0,0
1,No,2,507.5,1
2,No,3,517.5,2
3,No,4,510.0,3
4,Yes,1,502.5,4


###### Creating the final_data dataframe (steps 6 and 7)

In [14]:
# this code does the following:
# 1) renames the Price column to each dataframe's respective flight class
# 2) removes the unnecesary Class column
# 3) adds an index column (that is actually correct)
# 4) joins the dataframes based on their common index column

# step 6

reshaped_economy_and_premium_data = pd.merge(data_for_economy, data_for_premium, on='index')

reshaped_business_and_first_data = pd.merge(data_for_business, data_for_first, on='index')

reshaped_data_full = pd.merge(reshaped_economy_and_premium_data, reshaped_business_and_first_data, on='index')

# step 7

reshaped_data_full = reshaped_data_full.drop(reshaped_data_full.columns[[4, 5, 7, 8, 10, 11]], axis=1)

final_data = reshaped_data_full.rename(columns={'Flow Card?_x_x':'Flow Card?', 
                                                  'Quarter_x_x':'Quarter'})

# step 7

final_data.drop(columns=['index'], inplace=True)

final_data


,Flow Card?,Quarter,Economy,Premium,Business,First
0,No,1,204.0,241.2,515.00,1030.0
1,No,2,202.0,240.0,507.50,1000.0
2,No,3,201.0,240.0,517.50,1000.0
3,No,4,200.0,240.0,510.00,1015.0
4,Yes,1,201.0,249.6,502.50,1020.0
5,Yes,2,200.0,240.0,500.00,1020.0
6,Yes,3,206.0,241.2,502.50,1005.0
7,Yes,4,205.0,249.6,505.00,1030.0
8,No,1,438.0,574.8,1075.00,2340.0
9,No,2,445.0,553.8,1205.00,2325.0
